In [1]:
import pandas as pd

df = pd.read_excel("Nifty 50 Historical Data (1).xlsx")

df = df[['Date', 'Price']].copy()
df = df.sort_values(by='Date').reset_index(drop=True)

import numpy as np
df['log_return'] = np.log(df['Price'] / df['Price'].shift(1))
df = df.dropna().reset_index(drop=True)

In [2]:
from arch import arch_model

garch11 = arch_model(
    df['log_return'],
    mean='Constant',
    vol='GARCH',
    p=1,
    q=1,
    dist='normal'
)

garch_result = garch11.fit(disp='off')
garch_result.summary()

C:\Users\aronn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001059. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:             log_return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                8924.12
Distribution:                  Normal   AIC:                          -17840.2
Method:            Maximum Likelihood   BIC:                          -17816.7
                                        No. Observations:                 2682
Date:                Tue, Jan 06 2026   Df Residuals:                     2681
Time:                        22:37:30   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu         6.6348e-04  2.173e-05     30.537 8.407e-205 [6.209e-04,7.061e-04]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega      2.1181e-06  1.636e-11  1.295e+05      0.000 [2.118e-06,2.118e-06]
alpha[1]       0.1000  1.994e-02      5.014  5.332e-07   [6.091e-02,  0.139]
beta[1]        0.8800  1.488e-02     59.125      0.000     [  0.851,  0.909]
============================================================================

Covariance estimator: robust
"""

## One-Day Ahead Volatility

A one-day ahead conditional volatility forecast is obtained from the
estimated GARCH(1,1) model.

In [3]:
forecast = garch_result.forecast(horizon=1)
sigma_1day = np.sqrt(forecast.variance.iloc[-1, 0])

sigma_1day


np.float64(0.005570219564729474)

## Parametric VaR Computation

Value-at-Risk is computed using the Gaussian distribution assumption
and the forecasted conditional volatility.

In [4]:
from scipy.stats import norm

VaR_95 = -norm.ppf(0.05) * sigma_1day
VaR_99 = -norm.ppf(0.01) * sigma_1day

VaR_95, VaR_99


(np.float64(0.009162195853961328), np.float64(0.012958268442349109))

## Confidence Levels

VaR is evaluated at the 95% and 99% confidence levels to capture both
moderate and extreme downside risk.

## Portfolio Loss Interpretation

VaR estimates are translated into monetary terms for a hypothetical
portfolio to provide practical risk interpretation.

## Risk Interpretation

The VaR measures the maximum expected loss over one trading day under
normal market conditions at the specified confidence level.


In [5]:
portfolio_value = 1_000_000

loss_95 = VaR_95 * portfolio_value / 100
loss_99 = VaR_99 * portfolio_value / 100

loss_95, loss_99


(np.float64(91.62195853961327), np.float64(129.5826844234911))